In [1]:
get_ipython().run_line_magic('matplotlib', 'inline')
import os, sys

# sys.path.append("~/X/PythonForDataScience/air_traffic_project/submissions/first_real_submission/")

import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from problem import *
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, FunctionTransformer
from sklearn.decomposition import PCA
import geopy.distance
from sklearn.metrics import mean_squared_error
from model_selection_python import *
import statsmodels.api as sm
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance


from six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import os

from estimator import _merge_external_data
from keras.wrappers.scikit_learn import KerasRegressor

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from keras.metrics import RootMeanSquaredError

from xgboost import XGBRegressor
import xgboost as xgb


pd.set_option('display.max_columns', None)

In [7]:
X_train, y_train = get_train_data('..')
X_test, y_test = get_test_data('..')

X_train_merged = _merge_external_data(X_train)
X_test_merged = _merge_external_data(X_test)

# features_to_keep = ['WeeksToDeparture', 'week_mean', 'day_mean', 'month_mean', 'day_nb_mean', 
#                    'route_mean', 'std_wtd', 'n_days_departure']

X_train_data = X_train_merged #[features_to_keep]
X_test_data = X_test_merged
X_train_data.drop(['Departure', 'Arrival', 'holidays_dep', 'holidays_arr'], axis=1, inplace=True) 
X_test_data.drop(['Departure', 'Arrival', 'holidays_dep', 'holidays_arr'], axis=1, inplace=True)

/Users/Hugo/Desktop/X-HEC/Polytechnique/Semestre 1/Python_For_Data_Science/air_traffic_project/src/external_data.csv
/Users/Hugo/Desktop/X-HEC/Polytechnique/Semestre 1/Python_For_Data_Science/air_traffic_project/src/external_data.csv


In [8]:
print(X_train_data.shape)
print(X_test_data.shape)

(8902, 73)
(2226, 73)


In [15]:
#Keras Model
def build_nn():
    # create model
    model= Sequential(
                [Dense(73,activation='relu',input_shape=[73]),
                 Dense(36,activation='relu'),
                 Dropout(0.2),
                 Dense(18,activation='relu'),
                 Dense(9,activation='relu'),
                 Dense(1)
    ])
    # compile model
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['RootMeanSquaredError'])
    return model

# wrap the model using the function you created
keras_reg = KerasRegressor(
                build_nn,epochs=100,verbose=False)
keras_reg._estimator_type = "regressor"

# just create the pipeline
pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
    ('keras_reg',keras_reg)])

pipeline.fit(X_train_data, y_train)
# kfold = KFold(n_splits=10)
# results = cross_val_score(pipeline, X_train_data, y_train, cv=kfold)

Pipeline(steps=[('std_scaler', StandardScaler()),
                ('keras_reg',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x138d8f6d0>)])

In [16]:
# print("Standardized %.2f (%.2f) MSE" % (results.mean(), results.std()))
# print(results)

In [17]:
y_train_pred = pipeline.predict(X_train_data)
y_test_pred = pipeline.predict(X_test_data)

In [18]:
score_train = pipeline.score(X_train_data, y_train)
score_test = pipeline.score(X_test_data, y_test)

rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)

print(f"Training RMSE: {rmse_train} and Score: {score_train}")
print(f"Testing RMSE: {rmse_test} and Score: {score_test}")

Training RMSE: 0.6905525710210727 and Score: -0.4768628180027008
Testing RMSE: 0.7692071580888495 and Score: -0.5916798114776611


In [19]:
print(y_test_pred)
print(y_train_pred)

[10.476277 10.052585 10.223854 ... 11.114573  9.285128  8.954538]
[11.3927145  9.9986315 10.624433  ... 10.238039   9.757681   9.079252 ]
